# Python Script for: Continuous foraging behavior shapes patch-leaving decisions in pigeons: A 3D tracking study

Guillermo Hidalgo Gadea

Optimal foraging behavior is a key component of successful adaptations to natural environments. Understanding how animals decide to stay near food or to leave it for another food patch gives us insights into the underlying cognitive mechanisms that govern adaptive behaviors. 3D pose tracking was used to determine how pigeons exploit a 4 square meter arena with two separate platforms (i.e. food patches) whose absolute and relative elevations were manipulated. Detailed kinematic features of foraging and traveling behaviors were quantified using automated video tracking, without a need for manual coding. Our computational approach captured continuous, high-dimensional movement patterns and enabled precise quantification of travel costs between patches. Combined with mixed-effects survival analysis, our detailed behavioral tracking provided unprecedented insight into the moment-by-moment dynamics of patch-leaving decisions of pigeons. As expected from behavior optimization models, our results showed a preference to visit a ground food platform first, and longer latencies to leave an elevated platform. Foraging activity significantly decreased throughout a session, with shorter visits, less pecks per visit, and a decrease in inter-peck variability. However, a mixed-effects Cox regression modeled pigeons' patch-leaving probability, demonstrating that current and cumulative foraging parameters between patches significantly enhanced the model's predictive power beyond patch accessibility (i.e., beyond travel costs). This suggests that pigeons integrate both current environmental cues and their individual foraging history when making patch-leaving decisions. Our findings are discussed in relation to the marginal value theorem and optimal foraging theory.

`updated 22.08.2025`

## PART I: Demo Analysis

### Initialize Environment

In [ ]:
## Import Libraries ##
import os
import sys

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import entropy
np.random.seed(1234)

# load MotionPype
from motionpype import behaviorspecific, utils

### Read Project Directory

In [ ]:
def read_project_structure(projectpath):
    # find metadata
    metadatafile = utils.scrapdirbystring(os.path.dirname(projectpath), 'metadata.csv', output = False)[0]
    metadata = pd.read_csv(metadatafile, sep=',')
    metadata['date'] = [d.split('T')[0].split('-')[0]+d.split('T')[0].split('-')[1]+d.split('T')[0].split('-')[2] for d in metadata['date'].values]
    # find pose files
    pose_files = [file for file in utils.scrapdirbystring(projectpath, 'csv', output = False) if 'pose-3d' in file]
    sessions = []
    PIDs = []
    for file in pose_files:
        # get relative path to projectpath
        relpath = os.path.relpath(file, projectpath)
        dir = os.path.dirname(os.path.dirname(relpath))
        PIDs.append(os.path.basename(dir))
        sessions.append(os.path.dirname(dir))
    # print info
    print(f'Found {len(pose_files)} files in {projectpath}')
    print(f'Metadata found in {metadatafile}')
    print(f'with {len(set(sessions))} sessions: {set(sessions)}')
    print(f'from {len(set(PIDs))} Pigeons: {set(PIDs)}')

    return metadata, pose_files

In [ ]:
# set project parameters
projectpath = r"K:\ForagingPlatformsArena_local\Triangulation"
outputpath = r"K:\ForagingPlatformsArena_local\Analysis"

# read project structure
metadata, pose_files = read_project_structure(projectpath)

# set recording parameters (do not include paranthesis in bodypart string)
reference_points = ['cA','cB','cC','cD','cE','cF']
head = ['Head', 'UpperCere', 'LowerCere', 'BeakTip', 'LeftEye', 'RightEye', 'UpperNeck']
body = ['UpperSpine=LN', 'LowerSpine', 'MiddleSpine', 'UpperHalfSpine',
        'LowerHalfSpine', 'TailLeft', 'TailRight', 'TailCenter']

# Smoothing parameters
fps = 50 # in Hz

# Demo dataset
file = pose_files[10]	
# DEBUGGING
#file =  r'K:\ForagingPlatformsArena_local\Triangulation\20221101\P195\pose-3d\h265_crf12_20221101_ForagingPlatforms_P195.csv'
#file = r'K:\ForagingPlatformsArena_local\Triangulation\20230220\P122\pose-3d\h265_crf12_20230220_ForagingPlatforms_P122.csv'
PID = os.path.splitext(os.path.basename(file))[0].split('_')[-1]
date = os.path.splitext(os.path.basename(file))[0].split('_')[2]

### Read Session Metadata

In [ ]:
# get PID from file
def read_metadata(metadata, PID, date):
    # subset metadata
    idx1 = (metadata['pigeon_id']==PID).values
    idx2 = (metadata['date']==date).values

    entry = metadata[idx1&idx2]
    if len(entry) < 1:
        print('error, case not found!')
    elif len(entry) > 1:
        print('error, index not unique!')
    else:
        #extract metadata
        condition = entry['session_type.f'].values[0]
        session = entry['session_number'].values
        food_consumed = int(entry['food_depleted'] - entry['food_other'])
        depleted_A = int(entry['food_depleted_pos2'])
        depleted_B = int(entry['food_depleted_pos1'])
        depleted_high = food_consumed *0
        depleted_low = food_consumed *0
        if int(condition.split('-')[1]) >= 0:
            depleted_high += depleted_A
        if int(condition.split('-')[0]) >= 0:
            depleted_high += depleted_B
        if int(condition.split('-')[1]) <= 1: 
            depleted_low += depleted_A
        if int(condition.split('-')[0]) <= 1: 
            depleted_low += depleted_B
        
        sex = entry['sex'].values
        age = entry['age'].values
        colony = entry['colony'].values
        cohort = entry['cohort'].values
        normal_weight = entry['normal_weight'].values
        weight = entry['weight_g'].values

    # platform A on wall BC (left from door) and platform B on wall EF (right from door)
    elevation_A = int(condition.split('-')[1])*10
    elevation_B = int(condition.split('-')[0])*10

    return condition, session, food_consumed, depleted_A, depleted_B, depleted_high, depleted_low, sex, age, colony, cohort, normal_weight, weight, elevation_A, elevation_B


In [ ]:
condition, session, food_consumed, depleted_A, depleted_B, depleted_high, depleted_low, sex, age, colony, cohort, normal_weight, weight, elevation_A, elevation_B = read_metadata(metadata, PID, date)

### Track Pigeon over Time

In [ ]:
def track_pigeon(file, fps, reference_points, head, body):
    # read 3D data from csv file
    error, ncams, score, reference, pose = behaviorspecific.read_anipose_data(file, reference_points)

    # calculate stable centroids from filtered data
    behavior_smoothing = 300 # in ms, shortest meaningful behavior interval
    body_smooting =  int(np.ceil(behavior_smoothing/1000 *fps) // 2 * 2 + 1)
    filtered_head = behaviorspecific.median_offset_filtering(pose, head,  k_tracking = 41, max_dist = 60, interpolation = 'linear')
    centroid_head = behaviorspecific.reduceposetocentroid(filtered_head, head, med_smooth = 3)
    filtered_body = behaviorspecific.median_offset_filtering(pose, body,  k_tracking = 41, max_dist = 60, interpolation = 'linear')
    centroid_body = behaviorspecific.reduceposetocentroid(filtered_body, body, med_smooth = body_smooting)

    # calculate displacement as euclidean distance between frame-to-frame positions
    centroid_body_displacement = np.diff(centroid_body, n = 1, axis = 0, prepend = np.array([centroid_body.iloc[0,:]]))
    euclidean_body_displacement = np.sqrt(np.sum(centroid_body_displacement**2, axis = 1)) 
    centroid_head_displacement = np.diff(centroid_head, n = 1, axis = 0, prepend = np.array([centroid_head.iloc[0,:]]))
    euclidean_head_displacement = np.sqrt(np.sum(centroid_head_displacement**2, axis = 1))

    # save tracking to data frame
    centroid_head.columns=['centroid_head_x', 'centroid_head_y', 'centroid_head_z']
    centroid_body.columns=['centroid_body_x', 'centroid_body_y', 'centroid_body_z']
    df_tracking = pd.concat([centroid_head, centroid_body], axis=1)
    # append body displacement
    df_tracking['euclidean_head_disp'] = euclidean_head_displacement
    df_tracking['euclidean_body_disp'] = euclidean_body_displacement

    return df_tracking, pose, reference, filtered_head, filtered_body


In [ ]:
# TODO add visualization 
df_tracking, pose, reference, filtered_head, filtered_body = track_pigeon(file, fps, reference_points, head, body)
df_tracking

### Track Platform Visits

In [ ]:
def track_visits(df_tracking, reference, fps, elevation_A, elevation_B):
    # Platform ROI dimensions TODO
    width = 600 # in mm
    depth = 500
    height = 500 # padding comes on top
    padding = 50 # error around borders
    smooth_states = 2000 # in ms, shortest meaningful duration for a visit
    k_states =  int(np.ceil(smooth_states/1000 *fps) // 2 * 2 + 1)

    # extract centroid_head
    centroid_head = df_tracking[['centroid_head_x', 'centroid_head_y', 'centroid_head_z']]

    # track pigeon in platform ROI
    p1 = np.array(reference.loc[0,reference.columns.str.contains("cB")])
    p2 = np.array(reference.loc[0,reference.columns.str.contains("cC")])
    platform_A = behaviorspecific.findplatformROI(p1, p2, width, height, depth, padding, elevation_A)
    on_platform_A = behaviorspecific.findpointin3DROI(centroid_head, platform_A, smooth = k_states)

    q1 = np.array(reference.loc[0,reference.columns.str.contains("cE")])
    q2 = np.array(reference.loc[0,reference.columns.str.contains("cF")])
    platform_B = behaviorspecific.findplatformROI(q1, q2, width, height, depth, padding, elevation_B)
    on_platform_B = behaviorspecific.findpointin3DROI(centroid_head, platform_B, smooth =k_states)

    # save tracking to data frame
    df_tracking['on_platform'] = [a or b for a, b in zip(on_platform_A, on_platform_B)]
    df_tracking['on_platform_A'] = on_platform_A
    df_tracking['on_platform_B'] = on_platform_B

    # calculate transitions
    df_tracking['transition_in'] = 0
    df_tracking['transition_out']= 0
    df_tracking.loc[np.diff(df_tracking['on_platform'], prepend = 0)>0, 'transition_in'] = 1
    df_tracking.loc[np.diff(df_tracking['on_platform'], prepend = 0)<0, 'transition_out'] = 1
     
    # subset visits
    visits = df_tracking[df_tracking['transition_in']==1]
    frames = list(visits.index)

    # calculate self transitions
    df_tracking['transition_self'] = 0
    idx1 = np.diff(visits['on_platform_A'], prepend=True)*1
    idx2 = np.diff(visits['on_platform_B'], prepend=True)*1
    selftransitions = abs(idx2*idx1 -1)
    if len(selftransitions) > 0:
        selftransitions[0] = 0
    df_tracking.loc[frames, 'transition_self'] = selftransitions

    # assign platform identity
    df_tracking.loc[df_tracking['on_platform']==False, 'location'] = 'arena'
    df_tracking.loc[df_tracking['on_platform_A']==True, 'location'] = 'platform A'
    df_tracking.loc[df_tracking['on_platform_B']==True, 'location'] = 'platform B'

    # assign platform height
    df_tracking.loc[df_tracking['on_platform']==False, 'elevation'] = 0
    df_tracking.loc[df_tracking['on_platform_A']==True, 'elevation'] = elevation_A
    df_tracking.loc[df_tracking['on_platform_B']==True, 'elevation'] = elevation_B

    # calculate visit order
    df_tracking['visit_order'] = 0
    df_tracking['visit_order_A'] = 0
    df_tracking['visit_order_B'] = 0
    visit_order = np.cumsum(visits['on_platform'])
    visit_order_A = np.cumsum(visits['on_platform_A'])
    visit_order_B = np.cumsum(visits['on_platform_B'])
    df_tracking.loc[frames, 'visit_order'] = visit_order
    df_tracking.loc[frames, 'visit_order_A'] = visit_order_A
    df_tracking.loc[frames, 'visit_order_B'] = visit_order_B

    # combine order to current and alternative
    visits = df_tracking[df_tracking['transition_in']==1]
    df_tracking['current_order'] = 0
    df_tracking['alternative_order'] = 0
    current_order = np.where(visits['location'] == 'platform A', visits['visit_order_A'], visits['visit_order_B'])
    alternative_order = np.where(visits['location'] == 'platform A', visits['visit_order_B'], visits['visit_order_A'])
    df_tracking.loc[frames, 'current_order'] = current_order
    df_tracking.loc[frames, 'alternative_order'] = alternative_order

    # add visit order without transitions 
    df_tracking['visit_order_noself'] = 0
    visit_order_no_self = np.cumsum(visits['on_platform']-visits['transition_self'])
    df_tracking.loc[frames, 'visit_order_noself'] = visit_order_no_self

    # add visit status
    df_tracking['status'] = 0
    df_tracking.loc[frames, 'status'] = 1 # assume all visits are closed

    # calculate visit length
    df_tracking['visit_length'] = 0
    visit_length = df_tracking[df_tracking['transition_out']==1].index - df_tracking[df_tracking['transition_in']==1].index
    df_tracking.loc[frames, 'visit_length'] = [length/fps for length in visit_length]

    # calculate visit latency
    df_tracking['visit_latency'] = 0
    visit_latency = np.diff(df_tracking[df_tracking['transition_in']==1].index, prepend= 0)
    df_tracking.loc[frames, 'visit_latency'] = [latency/fps for latency in visit_latency]

    # calculate travel time
    df_tracking['travel_time'] = 0
    if len(visit_length) > 1:
        travel_time = list(visit_latency[1:] - visit_length[:-1])
        travel_time.insert(0, visit_latency[0])
    else:
        travel_time = visit_latency
    df_tracking.loc[frames, 'travel_time'] = [time/fps for time in travel_time]

    return df_tracking, platform_A, platform_B


In [ ]:
# TODO add visualization 
df_tracking, platform_A, platform_B = track_visits(df_tracking, reference, fps, elevation_A, elevation_B)
df_tracking

### Track Head Direction

In [ ]:
def track_head_direction(filtered_head, filtered_body, platform_A, platform_B):
    # TODO
    # calculate head direction
    headcenter_med, headvectors, headdirection_mean = behaviorspecific.calculateheadvectors(filtered_head, head, ksmooth = 1)
    
    # TODO calculate body angle

    # calculate direction of travel
    bodycenter_med, kinematicvec = behaviorspecific.kinematicvector(filtered_body, n = 25)

    # calculate projection angles towards palktforms
    ref_platform_A = platform_A[0] + platform_A[1]/2 - platform_A[3]/2
    ref_platform_B = platform_B[0] + platform_B[1]/2 - platform_B[3]/2
    head_angles_A = behaviorspecific.projectionangle(headcenter_med, headvectors, ref_platform_A)
    head_angles_B = behaviorspecific.projectionangle(headcenter_med, headvectors, ref_platform_B)
    kinematic_angles_A = behaviorspecific.projectionangle(bodycenter_med, kinematicvec, ref_platform_A)
    kinematic_angles_B = behaviorspecific.projectionangle(bodycenter_med, kinematicvec, ref_platform_B)

    # Add parameters to df_tracking

    return df_tracking

In [ ]:
# TODO add visualization 


### Track Pecking Behavior

In [ ]:
def track_pecking(df_tracking, fps, platform_A, platform_B):
    
    # extract centroid_head
    centroid_head = df_tracking[['centroid_head_x', 'centroid_head_y', 'centroid_head_z']]
    on_platform_A = list(df_tracking['on_platform_A'])
    on_platform_B = list(df_tracking['on_platform_B'])

    # use platform points 0,1,3 because 2 is the height
    distance_to_A = behaviorspecific.distance_to_plane(platform_A[0,:], platform_A[1,:], platform_A[3,:], centroid_head)
    distance_to_B = behaviorspecific.distance_to_plane(platform_B[0,:], platform_B[1,:], platform_B[3,:], centroid_head)

    # consider separating per platform here
    pecktimes_A = behaviorspecific.find_pecks(distance_to_A, fps)
    pecktimes_B = behaviorspecific.find_pecks(distance_to_B, fps)

    # check if pecks in pecktimes are on platform
    corrected_pecktimes_A = []
    for peck in pecktimes_A:
        if on_platform_A[peck]:
            corrected_pecktimes_A.append(peck)

    corrected_pecktimes_B = []
    for peck in pecktimes_B:
        if on_platform_B[peck]:
            corrected_pecktimes_B.append(peck)

    pecktimes = corrected_pecktimes_A + corrected_pecktimes_B

    # extract peck positions
    peckpositions = centroid_head.iloc[pecktimes]
    peckpositions.columns=['peckposition_x', 'peckposition_y', 'peckposition_z']

    # calculate distance to platform
    distance_to_B[~np.array(on_platform_B)] = np.nan
    distance_to_A[~np.array(on_platform_A)] = np.nan
    distance_to_platform = np.where(np.isnan(distance_to_A), distance_to_B, distance_to_A)

    # save tracking to data frame
    df_tracking['distance_to_platform'] = distance_to_platform
    df_tracking['peck'] = 0
    df_tracking.loc[pecktimes,'peck'] = 1
    df_tracking = pd.concat([df_tracking, peckpositions], axis=1, join='outer')

    return df_tracking

In [ ]:
df_tracking = track_pecking(df_tracking, fps, platform_A, platform_B)
df_tracking

### Feature Aggregation

In [ ]:
def process_session_pervisits(df_tracking, fps):
    '''
    Return pandas DF with aggregate statistics for each session
    '''

    # specify percentage of visits to add as censor
    p_censor = 0.5
    IPI_bins = [.46, 1.1, 1.9]
    IPD_bins = [55, 165, 260]

    # get transition indices
    idx_in = df_tracking[df_tracking['transition_in']==1].index
    idx_out = df_tracking[df_tracking['transition_out']==1].index
    idx_out_pre = np.insert(idx_out,0,0)

    # calculate visit aggregates 
    visit_head_disp = []
    visit_head_speed = []
    visit_peck_count = []
    visit_peck_rate = []
    visit_IPI_entropy = []
    visit_IPI_RMSSD = []
    visit_IPD_entropy = []
    visit_IPD_RMSSD = []

    for a, b in zip(idx_in, idx_out):
        visit = df_tracking.loc[a:b]
        # head displacement
        head_disp = np.sum(visit['euclidean_head_disp'])
        visit_head_disp.append(head_disp)
        # head speed
        head_speed = np.mean(np.abs(np.diff(visit['euclidean_head_disp'])))*fps
        visit_head_speed.append(head_speed)
        # peck count
        peck_count = np.sum(visit['peck'])
        visit_peck_count.append(int(peck_count))
        # normalized peck rate
        peck_rate = peck_count/len(visit)*fps
        visit_peck_rate.append(peck_rate)
        if peck_count > 2:
            # Inter peck interval
            ipi = np.diff(visit[visit['peck']==1].index) / fps # in sec
            # calculate RMSSD
            visit_IPI_RMSSD.append(np.sqrt(np.mean(ipi**2)))
            # quantize IPI
            IPI_steps = np.digitize(ipi, IPI_bins)
            value,counts = np.unique(IPI_steps, return_counts=True)
            # calculate entropy
            visit_IPI_entropy.append(entropy(counts, base=None))
            # Inter peck distance
            peckposition = visit[visit['peck']==1][visit.columns[visit.columns.str.contains('peckposition')]]
            ipd = np.sqrt(np.sum(np.diff(peckposition, axis=0)**2, axis=1)) # in mm
            # calculate RMSSD
            visit_IPD_RMSSD.append(np.sqrt(np.mean(ipd**2)))
            # quantize IPD
            IPD_steps = np.digitize(ipd, IPD_bins)
            values, counts = np.unique(IPD_steps, return_counts=True)
            # calculate entropy
            visit_IPD_entropy.append(entropy(counts, base=None))
        else:
            visit_IPI_entropy.append(0)
            visit_IPI_RMSSD.append(0)
            visit_IPD_entropy.append(0)
            visit_IPD_RMSSD.append(0)
    
    # calculate travel aggregates
    travel_body_disp = []
    travel_body_speed = []
    for c, d in zip(idx_out_pre, idx_in):
        travel = df_tracking.loc[c:d]
        # body displacement
        body_disp = np.sum(travel['euclidean_body_disp'])
        travel_body_disp.append(body_disp)
        # calculate travel speed
        body_speed = np.mean(abs(np.diff(travel['euclidean_body_disp'])))*fps
        travel_body_speed.append(body_speed)
        # TODO
        # calcualte travel directedness (see kinematic vector)
        # calculate haed direction 
    
    # add parameters to df
    df_tracking.loc[idx_in, 'head_disp'] = visit_head_disp
    df_tracking.loc[idx_in, 'head_speed'] = visit_head_speed
    df_tracking.loc[idx_in, 'peck_count'] = visit_peck_count
    df_tracking.loc[idx_in, 'peck_rate'] = visit_peck_rate
    df_tracking.loc[idx_in, 'IPI_entropy'] = visit_IPI_entropy
    df_tracking.loc[idx_in, 'IPI_rmssd'] = visit_IPI_RMSSD
    df_tracking.loc[idx_in, 'IPD_entropy'] = visit_IPD_entropy
    df_tracking.loc[idx_in, 'IPD_rmssd'] = visit_IPD_RMSSD
    df_tracking.loc[idx_in, 'travel_dist'] = travel_body_disp
    df_tracking.loc[idx_in, 'travel_speed'] = travel_body_speed

    # Filter visits
    visits = df_tracking.loc[idx_in]

    # Add cummulative parameters from last platform, alternative
    cum_visit_length = []
    cum_visits = []
    cum_head_disp = []
    cum_peck_count = []
    cum_travel_dist = []
    cum_travel_time = []
    alt_cum_visit_length = []
    alt_cum_visits = []
    alt_cum_head_disp = []
    alt_cum_peck_count = []
    alt_cum_travel_dist = []
    alt_cum_travel_time = []

    for visit in visits.index:
        previous_visits = visits.loc[:visit-1]
        current_visits = visits.loc[:visit]
        # check if current visit is platform A or platform B
        if visits.loc[visit,'on_platform_A']:
            # assign history of alternative patch
            alt_prev_visits = previous_visits[previous_visits['on_platform_B']]
            curr_prev_visits = current_visits[current_visits['on_platform_A']]
        else:
            alt_prev_visits = previous_visits[previous_visits['on_platform_A']]
            curr_prev_visits = current_visits[current_visits['on_platform_B']]

        if alt_prev_visits.empty:
            cum_visit_length.append(np.max(np.cumsum(curr_prev_visits['visit_length'])))
            cum_visits.append(len(curr_prev_visits))
            cum_head_disp.append(np.max(np.cumsum(curr_prev_visits['head_disp'])))
            cum_peck_count.append(np.max(np.cumsum(curr_prev_visits['peck_count'])))
            cum_travel_dist.append(np.max(np.cumsum(curr_prev_visits['travel_dist'])))
            cum_travel_time.append(np.max(np.cumsum(curr_prev_visits['travel_time'])))
            # replace with 0 for empty history
            alt_cum_visit_length.append(0)
            alt_cum_visits.append(0)
            alt_cum_head_disp.append(0)
            alt_cum_peck_count.append(0)
            alt_cum_travel_dist.append(0)
            alt_cum_travel_time.append(0)
        else:
            cum_visit_length.append(np.max(np.cumsum(curr_prev_visits['visit_length'])))
            cum_visits.append(len(curr_prev_visits))
            cum_head_disp.append(np.max(np.cumsum(curr_prev_visits['head_disp'])))
            cum_peck_count.append(np.max(np.cumsum(curr_prev_visits['peck_count'])))
            cum_travel_dist.append(np.max(np.cumsum(curr_prev_visits['travel_dist'])))
            cum_travel_time.append(np.max(np.cumsum(curr_prev_visits['travel_time'])))
            alt_cum_visit_length.append(np.max(np.cumsum(alt_prev_visits['visit_length'])))
            alt_cum_visits.append(len(alt_prev_visits))
            alt_cum_head_disp.append(np.max(np.cumsum(alt_prev_visits['head_disp'])))
            alt_cum_peck_count.append(np.max(np.cumsum(alt_prev_visits['peck_count'])))
            alt_cum_travel_dist.append(np.max(np.cumsum(alt_prev_visits['travel_dist'])))
            alt_cum_travel_time.append(np.max(np.cumsum(alt_prev_visits['travel_time'])))

    # assign history on alternative patch
    visits['cum_visit_length'] = cum_visit_length
    visits['cum_visits'] = cum_visits
    visits['cum_head_disp'] = cum_head_disp
    visits['cum_peck_count'] = cum_peck_count
    visits['cum_travel_dist'] = cum_travel_dist
    visits['cum_travel_time'] = cum_travel_time
    visits['alt_cum_visit_length'] = alt_cum_visit_length
    visits['alt_cum_visits'] = alt_cum_visits
    visits['alt_cum_head_disp'] = alt_cum_head_disp
    visits['alt_cum_peck_count'] = alt_cum_peck_count
    visits['alt_cum_travel_dist'] = alt_cum_travel_dist
    visits['alt_cum_travel_time'] = alt_cum_travel_time
    
    # Add censoring for unfinished visits
    # subset 50% of longest visits on each platform
    visits_on_A = visits[visits['on_platform_A'] == 1]
    visits_on_B = visits[visits['on_platform_B'] == 1]
    filtered_visits_A = visits_on_A[visits_on_A['visit_length'] > visits_on_A['visit_length'].median()]
    filtered_visits_B = visits_on_B[visits_on_B['visit_length'] > visits_on_B['visit_length'].median()]
    combined_visits = pd.concat([filtered_visits_A, filtered_visits_B]).index
    # subset random 25% of total visits (50% of 50%, filtered for longer half and balanced between platforms)
    censored_percent = p_censor /0.5 # only considering top 50% of visits
    idx = np.random.choice(combined_visits, int(round(len(combined_visits)*censored_percent)), replace=False)
    censored_visits = visits.loc[idx]
    censored_visits['status'] = 0
    
    # reduce visit length randomly between .25 and .75
    rand_reduce = np.random.uniform(.25, .75, len(censored_visits))
    censored_visits['visit_length'] = censored_visits['visit_length'] - censored_visits['visit_length'] * rand_reduce
    # recalculate parameters for censored visits
    idx_in_cens = censored_visits.index
    idx_out_cens = round(idx_in_cens + censored_visits['visit_length']*fps).astype(int).values
    censored_head_disp = []
    censored_head_speed = []
    censored_peck_count = []
    censored_peck_rate = []
    censored_IPI_entropy = []
    censored_IPI_RMSSD = []
    censored_IPD_entropy = []
    censored_IPD_RMSSD = []

    for a, b in zip(idx_in_cens, idx_out_cens):
        visit = df_tracking.loc[a:b]
        # head displacement
        head_disp = np.sum(visit['euclidean_head_disp'])
        censored_head_disp.append(head_disp)
        # head speed
        head_speed = np.mean(np.abs(np.diff(visit['euclidean_head_disp'])))*fps
        censored_head_speed.append(head_speed)
        # peck count
        peck_count = np.sum(visit['peck'])
        censored_peck_count.append(int(peck_count))
        # normalized peck rate
        peck_rate = peck_count/len(visit)*fps
        censored_peck_rate.append(peck_rate)
        if peck_count > 2:
             # Inter peck interval
            ipi = np.diff(visit[visit['peck']==1].index) / fps # in sec
            # calculate RMSSD
            censored_IPI_RMSSD.append(np.sqrt(np.mean(ipi**2)))
            # quantize IPI
            IPI_steps = np.digitize(ipi, IPI_bins)
            value,counts = np.unique(IPI_steps, return_counts=True)
            # calculate entropy
            censored_IPI_entropy.append(entropy(counts, base=None))
            # Inter peck distance
            peckposition = visit[visit['peck']==1][visit.columns[visit.columns.str.contains('peckposition')]]
            ipd = np.sqrt(np.sum(np.diff(peckposition, axis=0)**2, axis=1)) # in mm
            # calculate RMSSD
            censored_IPD_RMSSD.append(np.sqrt(np.mean(ipd**2)))
            # quantize IPD
            IPD_steps = np.digitize(ipd, IPD_bins)
            values, counts = np.unique(IPD_steps, return_counts=True)
            # calculate entropy
            censored_IPD_entropy.append(entropy(counts, base=None))                                      
        else: 
            censored_IPD_entropy.append(0)
            censored_IPD_RMSSD.append(0)
            censored_IPI_entropy.append(0)
            censored_IPI_RMSSD.append(0)

    
    # recalculate cummulative features for censored visits
    cum_visit_length = []
    cum_visits = []
    cum_head_disp = []
    cum_peck_count = []
    cum_travel_dist = []
    cum_travel_time = []

    for visit in censored_visits.index:
        current_visits = censored_visits.loc[:visit]
        # check if current visit is platform A or platform B
        if censored_visits.loc[visit,'on_platform_A']:
            # assign history of alternative patch
            curr_prev_visits = current_visits[current_visits['on_platform_A']]
        else:
            curr_prev_visits = current_visits[current_visits['on_platform_B']]

        cum_visit_length.append(np.max(np.cumsum(curr_prev_visits['visit_length'])))
        cum_visits.append(len(curr_prev_visits))
        cum_head_disp.append(np.max(np.cumsum(curr_prev_visits['head_disp'])))
        cum_peck_count.append(np.max(np.cumsum(curr_prev_visits['peck_count'])))
        cum_travel_dist.append(np.max(np.cumsum(curr_prev_visits['travel_dist'])))
        cum_travel_time.append(np.max(np.cumsum(curr_prev_visits['travel_time'])))

    # add to censored visits
    censored_visits['head_disp'] = censored_head_disp
    censored_visits['head_speed'] = censored_head_speed
    censored_visits['peck_count'] = censored_peck_count
    censored_visits['peck_rate'] = censored_peck_rate
    censored_visits['IPI_entropy'] = censored_IPI_entropy
    censored_visits['IPI_rmssd'] = censored_IPI_RMSSD
    censored_visits['IPD_entropy'] = censored_IPD_entropy
    censored_visits['IPD_rmssd'] = censored_IPD_RMSSD
    censored_visits['cum_visit_length'] = cum_visit_length
    censored_visits['cum_visits'] = cum_visits
    censored_visits['cum_head_disp'] = cum_head_disp
    censored_visits['cum_peck_count'] = cum_peck_count
    censored_visits['cum_travel_dist'] = cum_travel_dist
    censored_visits['cum_travel_time'] = cum_travel_time

    # join visits
    visits = pd.concat([visits, censored_visits]).sort_index()

    return df_tracking, visits

In [ ]:
df_tracking, visits = process_session_pervisits(df_tracking, fps)
visits

## PART 2: Data Processing

In [ ]:
# set project parameters
projectpath = r"K:\ForagingPlatformsArena_local\Triangulation"
outputpath = r"K:\ForagingPlatformsArena_local\Analysis"

# read project structure
metadata, pose_files = read_project_structure(projectpath)

# set recording parameters (do not include paranthesis in bodypart string)
reference_points = ['cA','cB','cC','cD','cE','cF']
head = ['Head', 'UpperCere', 'LowerCere', 'BeakTip', 'LeftEye', 'RightEye', 'UpperNeck']
body = ['UpperSpine=LN', 'LowerSpine', 'MiddleSpine', 'UpperHalfSpine',
        'LowerHalfSpine', 'TailLeft', 'TailRight', 'TailCenter']

# Smoothing parameters
fps = 50 # in Hz
# NOTE first three videos on 20221019 are recorded at 100Hz 

for file in pose_files:
    print(f'Processing file {file}')
    PID = os.path.splitext(os.path.basename(file))[0].split('_')[-1]
    date = os.path.splitext(os.path.basename(file))[0].split('_')[2]
    if date == '20221019':
        fps = 100
        print('corrected fps')
    else:
        fps = 50
    # read metadata
    condition, session, food_consumed, depleted_A, depleted_B, depleted_high, depleted_low, sex, age, colony, cohort, normal_weight, weight, elevation_A, elevation_B = read_metadata(metadata, PID, date)
    print(f'Pigeon: {PID}, Date: {date}, Session: {session}, Condition: {condition}, Food consumed: {food_consumed}')
    # track pigeon
    df_tracking, pose, reference, filtered_head, filtered_body = track_pigeon(file, fps, reference_points, head, body)
    # track platform visits
    df_tracking, platform_A, platform_B = track_visits(df_tracking, reference, fps, elevation_A, elevation_B)
    # track pecking behavior
    df_tracking = track_pecking(df_tracking, fps, platform_A, platform_B)
    # aggregate features
    df_tracking, visits = process_session_pervisits(df_tracking, fps)
    # add metadata to processed data
    visits['PID'] = PID
    visits['date'] = date
    visits['session'] = session[0]
    visits['condition'] = condition
    visits['sex'] = sex[0]
    visits['age'] = age[0]
    visits['colony'] = colony[0]
    visits['cohort'] = cohort[0]
    visits['normal_weight'] = normal_weight[0]
    visits['weight'] = weight[0]
    visits['food_consumed'] = food_consumed
    visits['depleted_A'] = depleted_A
    visits['depleted_B'] = depleted_B
    visits['depleted_high'] = depleted_high
    visits['depleted_low'] = depleted_low
    visits['elevation_A'] = elevation_A
    visits['elevation_B'] = elevation_B
    visits['total_latency'] = visits.index/fps
    # change order of columns
    visits = visits[['date', 'PID', 'session', 'condition', 'sex', 'age', 'colony', 'cohort', 'normal_weight', 'weight', 
                     'food_consumed', 'depleted_A', 'depleted_B', 'depleted_high', 'depleted_low', 'status',
                     'visit_order', 'visit_order_noself', 'current_order', 'alternative_order', 'location', 'elevation', 
                     'transition_self', 'visit_length', 'visit_latency', 'total_latency', 'travel_time', 'head_disp', 'head_speed',
                     'peck_count', 'peck_rate', 'IPI_entropy', 'IPI_rmssd', 'IPD_entropy', 'IPD_rmssd',
                     'travel_dist', 'travel_speed', 'cum_visit_length', 'cum_visits', 'cum_head_disp', 
                     'cum_peck_count', 'cum_travel_dist', 'cum_travel_time', 'alt_cum_visit_length', 'alt_cum_visits', 
                     'alt_cum_head_disp', 'alt_cum_peck_count', 'alt_cum_travel_dist', 'alt_cum_travel_time']]
       
    # save processed data
    filename = os.path.join(outputpath,os.path.splitext(os.path.basename(file))[0] + '_visits.csv')
    visits.to_csv(filename, sep=',', index=False)
    # save df tracking
    filename = os.path.join(outputpath,os.path.splitext(os.path.basename(file))[0] + '_tracking.pkl')
    df_tracking.to_pickle(filename)
    
    print(f'Data saved!')


In [ ]:
# aggregate all files into single dataset
processed_files = utils.scrapdirbystring(outputpath, '_visits.csv', output = False)

# read all csv files and merge into single dataset
dataset = pd.DataFrame()
for file in processed_files:
    df = pd.read_csv(file)
    dataset = pd.concat([dataset, df], ignore_index=True)

# save aggregate
filename = os.path.join(outputpath,'AggregateVisits.csv')
dataset.to_csv(filename, sep=',', index=False)

## PART 3: Data Analysis

#### Hypothesis: 
More exploratory behavior towards the end, variability of behavior: When outside of a patch, the pigeons should move toward patch more often in the first third of a session and move away from patch more often in the last third of a session. Head movement and locomotion variability lower during time on task.

In [ ]:
# read tracking data
projectpath = r"K:\ForagingPlatformsArena_local\Triangulation"
outputpath = r"K:\ForagingPlatformsArena_local\Analysis"
tracked_files = utils.scrapdirbystring(outputpath, 'tracking.pkl', output = False)

In [ ]:
df_tracking = pd.read_pickle(tracked_files[5])
df_tracking

### Compute spatiotemporal bins for IPI and IPD entropy

In [ ]:
# read all data and merge ipi and ipd
IPI = []
IPD = []
for file in tracked_files:
    date = os.path.splitext(os.path.basename(file))[0].split('_')[2]
    if date == '20221019':
        fps = 100
        print('corrected fps')
    else:
        fps = 50
    df_tracking = pd.read_pickle(file)
    idx_in = df_tracking[df_tracking['transition_in']==1].index
    idx_out = df_tracking[df_tracking['transition_out']==1].index
    for a, b in zip(idx_in, idx_out):
        visit = df_tracking.loc[a:b]
        peck_count = np.sum(visit['peck'])
        if peck_count > 1:
            # Inter peck interval
            ipi = np.diff(visit[visit['peck']==1].index) / fps
            # Inter peck distance
            peckposition = visit[visit['peck']==1][visit.columns[visit.columns.str.contains('peckposition')]]
            ipd = np.sqrt(np.sum(np.diff(peckposition, axis=0)**2, axis=1))
            # save
            IPI.append(ipi)
            IPD.append(ipd)
# flatten list of lists
IPI = [item for sublist in IPI for item in sublist]
IPD = [item for sublist in IPD for item in sublist]

#### IPD boundries based on hole distances

In [ ]:
fig = plt.figure(figsize=(10,5))
plt.rcParams.update({'font.size': 20})
plt.hist(IPD, bins = 200);
plt.ylim(0, 800)
plt.axvline(10, color='k', linestyle='--') # same hole
plt.axvline(100, color='r', linestyle='--') # one hole cross
plt.axvline(140, color='r', linestyle='--') # one hole diagonal
plt.axvline(200, color='g', linestyle='--') # two holes cross
plt.axvline(240, color='g', linestyle='--') # one holes cross one diagonal
plt.axvline(280, color='g', linestyle='--') # two holes diagonal
plt.axvline(300, color='b', linestyle='--') # more than two holes
plt.axvline(780, color='b', linestyle='--') # max platform diagonal
# minima boundries
plt.axvline(55, color='k', linestyle='-')
plt.axvline(165, color='k', linestyle='-')
plt.axvline(260, color='k', linestyle='-')

plt.xlabel('Inter-Peck Distance (mm)')
plt.ylabel('Frequency')

plt.show()


In [ ]:
IPD_bins = [55, 165, 260]
IPD_steps = np.digitize(IPD, IPD_bins)
values, counts = np.unique(IPD_steps, return_counts=True)

#### IPI boundries based on peck times

In [ ]:
fig = plt.figure(figsize=(10,5))
plt.rcParams.update({'font.size': 20})
plt.hist([x for x in IPI if x < 5], bins = 500);
plt.axvline(0.2, color='k', linestyle='-') # lower boundry
plt.axvline(0.31, color='r', linestyle='--') # expected by juan delius 2003 (https://kops.uni-konstanz.de/server/api/core/bitstreams/8dae99c9-0774-412d-8086-61902f498130/content)
plt.axvline(0.31*2, color='r', linestyle='--') # double expected
plt.axvline(0.31*3, color='r', linestyle='--') # triple expected
plt.axvline(0.31*4, color='r', linestyle='--') # quadruple expected
plt.axvline(0.46, color='k', linestyle='-') # boundry more than 1x
plt.axvline(1.1, color='k', linestyle='-') # boundry more than 3x
plt.axvline(1.9, color='k', linestyle='-') # boundry more than 6x

plt.xlabel('Inter-Peck Interval (sec)')
plt.ylabel('Frequency')
plt.show()


In [ ]:
IPI_bins = [.46, 1.1, 1.9]
IPI_steps = np.digitize(IPI, IPI_bins)
values, counts = np.unique(IPI_steps, return_counts=True)

In [ ]:
IPI_H = []
IPD_H = []
IPI_RMSSD = []
IPD_RMSSD = []
for file in tracked_files:
    df_tracking = pd.read_pickle(file)
    IPI_H.append(df_tracking['IPI_entropy'])
    IPD_H.append(df_tracking['IPD_entropy'])
    IPI_RMSSD.append(df_tracking['IPI_rmssd'])
    IPD_RMSSD.append(df_tracking['IPD_rmssd'])
# flatten list of lists
IPI_H = [item for sublist in IPI_H for item in sublist]
IPD_H = [item for sublist in IPD_H for item in sublist]
IPI_RMSSD = [item for sublist in IPI_RMSSD for item in sublist]
IPD_RMSSD = [item for sublist in IPD_RMSSD for item in sublist]

# exclude nan values
IPI_H = [x for x in IPI_H if not np.isnan(x)]
IPD_H = [x for x in IPD_H if not np.isnan(x)]
IPI_RMSSD = [x for x in IPI_RMSSD if not np.isnan(x)]
IPD_RMSSD = [x for x in IPD_RMSSD if not np.isnan(x)]

In [ ]:
# plt scatterplot of IPI_H vs IPD_H
fig = plt.figure(figsize=(10,10))
plt.rcParams.update({'font.size': 20})
# add jitter to scatter
x = [a + np.random.normal(0, 0.02) for a in IPI_H]
y = [b + np.random.normal(0, 0.02) for b in IPD_H]
plt.scatter(x, y, alpha=0.5, s = 5)
plt.xlabel('IPI Entropy')
plt.ylabel('IPD Entropy')
plt.show()


In [ ]:
# plt scatterplot of IPI_H vs IPD_H
fig = plt.figure(figsize=(10,10))
plt.rcParams.update({'font.size': 20})
# add jitter to scatter
x = [a + np.random.normal(0, 0.02) for a in IPI_RMSSD]
y = [b + np.random.normal(0, 0.02) for b in IPD_RMSSD]
plt.scatter(x, y, alpha=0.5, s = 5)
plt.xlim(0, 10)
plt.xlabel('IPI RMSSD')
plt.ylabel('IPD RMSSD')
plt.show()

### Time away from platform

In [ ]:
from scipy import interpolate

# Create a function to resample a signal to a common length
def resample_signal(signal, new_length):
    old_indices = np.linspace(0, len(signal)-1, num=len(signal))
    new_indices = np.linspace(0, len(signal)-1, num=new_length)
    interpolator = interpolate.interp1d(old_indices, signal)
    return interpolator(new_indices)

In [ ]:
timeonplatform = pd.DataFrame()
excluded = ['_P122_', '_P791_', '_P589_']
for file in tracked_files:
    date = os.path.splitext(os.path.basename(file))[0].split('_')[2]
    if date == '20221019':
        fps = 100
        print('corrected fps')
    else:
        fps = 50
    if any(x in file for x in excluded):
        continue
    # get metadata such as condition, session, food consumed, etc
    PID = os.path.splitext(os.path.basename(file))[0].split('_')[-2]
    condition, session, food_consumed, depleted_A, depleted_B, depleted_high, depleted_low, sex, age, colony, cohort, normal_weight, weight, elevation_A, elevation_B = read_metadata(metadata, PID, date)    # append to timeonplatform dataframe with metadata
    if condition == '75-0':
        condition = '0-75'
    if food_consumed < 1:
        continue
    if condition == '0-75':
        if depleted_A < 1:
            continue
        if depleted_B < 1:
            continue
    # read tracking data
    df_tracking = pd.read_pickle(file)
    # calculate cumsum of time on and off platform
    cumsum_onplatform = np.cumsum(df_tracking['on_platform'])/fps
    cumsum_offplatform = np.cumsum(~df_tracking['on_platform'])/fps
    # resample to same length
    cumsum_onplatform = resample_signal(cumsum_onplatform, 60000)
    cumsum_offplatform = resample_signal(cumsum_offplatform, 60000)
    # normalized cumsums
    norm_cumsum_onplatform = cumsum_onplatform/(np.nanmax(cumsum_onplatform))
    norm_cumsum_offplatform = cumsum_offplatform/(np.nanmax(cumsum_offplatform))
    # calculate area under curve
    AUC_onplatform = np.trapz(norm_cumsum_onplatform)
    AUC_onplatform_firstthird = np.trapz(norm_cumsum_onplatform[0:len(cumsum_onplatform)//3])
    AUC_onplatform_lastthird = np.trapz(norm_cumsum_onplatform[2*len(cumsum_onplatform)//3:len(cumsum_onplatform)])
    AUC_offplatform = np.trapz(norm_cumsum_offplatform)
    AUC_offplatform_firstthird = np.trapz(norm_cumsum_offplatform[0:len(cumsum_offplatform)//3])
    AUC_offplatform_lastthird = np.trapz(norm_cumsum_offplatform[2*len(cumsum_offplatform)//3:len(cumsum_offplatform)])
    ABC = AUC_onplatform - AUC_offplatform
    ABC_firstthird = AUC_onplatform_firstthird - AUC_offplatform_firstthird
    ABC_lastthird = AUC_onplatform_lastthird - AUC_offplatform_lastthird
    # calculate area under curve from diagonal
    diagonal = np.linspace(0, 1, num=60000)
    AUC_reference = np.trapz(diagonal)
    AUC_reference_firstthird = np.trapz(diagonal[0:len(diagonal)//3])
    AUC_reference_lastthird = np.trapz(diagonal[2*len(diagonal)//3:len(diagonal)])
    AUC_onplatform_ref = AUC_onplatform / AUC_reference
    AUC_onplatform_firstthird_ref = AUC_onplatform_firstthird / AUC_reference_firstthird
    AUC_onplatform_lastthird_ref = AUC_onplatform_lastthird / AUC_reference_lastthird
    AUC_offplatform_ref = AUC_offplatform / AUC_reference
    AUC_offplatform_firstthird_ref = AUC_offplatform_firstthird / AUC_reference_firstthird
    AUC_offplatform_lastthird_ref = AUC_offplatform_lastthird / AUC_reference_lastthird

    # save in loop
    timeonplatform = timeonplatform.append({'PID': PID, 'date': date, 'condition': condition, 'session': int(session), 'performance': float(food_consumed/60), 'age': int(age), 'normal_weight': int(normal_weight), 'deprivation': float(weight/normal_weight),
                                            'AUC_onplatform_ref': AUC_onplatform_ref, 'AUC_offplatform_ref': AUC_offplatform_ref, 'AUC_onplatform_firstthird_ref': AUC_onplatform_firstthird_ref, 'AUC_offplatform_firstthird_ref': AUC_offplatform_firstthird_ref,
                                            'AUC_onplatform_lastthird_ref': AUC_onplatform_lastthird_ref, 'AUC_offplatform_lastthird_ref': AUC_offplatform_lastthird_ref, 'cumsum_onplatform': cumsum_onplatform, 'cumsum_offplatform': cumsum_offplatform,
                                            'norm_cumsum_onplatform': norm_cumsum_onplatform, 'norm_cumsum_offplatform': norm_cumsum_offplatform}, ignore_index=True)

In [ ]:
# transform data to aggregate on/off platform and first/last timepoint
timeonplatform_long = pd.melt(timeonplatform, id_vars=['PID', 'date', 'condition', 'session', 'performance', 'age', 'normal_weight', 'deprivation'], 
                              value_vars=['AUC_onplatform_firstthird_ref', 'AUC_offplatform_firstthird_ref', 'AUC_onplatform_lastthird_ref', 'AUC_offplatform_lastthird_ref'], 
                              var_name='metric', value_name='AUC_norm')
timeonplatform_long['timepoint'] = ['first 3rd' for x in timeonplatform_long['metric'] if 'firstthird' in x] + ['last 3rd' for x in timeonplatform_long['metric'] if 'last' in x]
timeonplatform_long['allocation'] = ['on platform' if 'onplatform' in x else 'off platform' for x in timeonplatform_long['metric']]
timeonplatform_long = timeonplatform_long.drop(columns=['metric'])
timeonplatform_long
# save as csv
filename = os.path.join(outputpath,'Timeallocation.csv')
timeonplatform_long.to_csv(filename, sep=',', index=False)

#### Absolute cumulative time

In [ ]:
# Absolute 
plt.rcParams.update({'font.size': 20})
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(21,7))
for condition in set(timeonplatform['condition']):
    # calculate mean and sem for each condition
    subset = timeonplatform[timeonplatform['condition'] == condition]
    unpacked_onplatform = list(zip(*subset['cumsum_onplatform']))
    mean_onplatform = [np.mean(item) for item in unpacked_onplatform]
    std_onplatform = [np.std(item) for item in unpacked_onplatform]
    sem_onplatform = [std/np.sqrt(len(subset['cumsum_onplatform'])) for std in std_onplatform]
    unpacked_offplatform = list(zip(*subset['cumsum_offplatform']))
    mean_offplatform = [np.mean(item) for item in unpacked_offplatform]
    std_offplatform = [np.std(item) for item in unpacked_offplatform]
    sem_offplatform = [std/np.sqrt(len(subset['cumsum_offplatform'])) for std in std_offplatform]
    mean_max = (np.max(mean_onplatform) + np.max(mean_offplatform))/2
    if condition == '0-0':
        # plot mean with ribbon in axis
        ax1.plot(mean_onplatform, label='on platform', color = 'r')
        ax1.fill_between(range(len(mean_onplatform)), np.array(mean_onplatform) - 1.96*np.array(sem_onplatform), np.array(mean_onplatform) + 1.96*np.array(sem_onplatform), alpha=0.3, color = 'r')
        ax1.plot(mean_offplatform, label = 'off platform', color = 'b')
        ax1.fill_between(range(len(mean_offplatform)), np.array(mean_offplatform) - 1.96*np.array(sem_offplatform), np.array(mean_offplatform) + 1.96*np.array(sem_offplatform), alpha=0.3, color = 'b')
        # plot reference line from 0 to mean_max
        ax1.plot([0,60000], [0,mean_max], 'k--')
    elif condition == '0-75':
        ax2.plot(mean_onplatform, label='on platform', color = 'r')
        ax2.fill_between(range(len(mean_onplatform)), np.array(mean_onplatform) - 1.96*np.array(sem_onplatform), np.array(mean_onplatform) + 1.96*np.array(sem_onplatform), alpha=0.3, color = 'r')
        ax2.plot(mean_offplatform, label = 'off platform', color = 'b')
        ax2.fill_between(range(len(mean_offplatform)), np.array(mean_offplatform) - 1.96*np.array(sem_offplatform), np.array(mean_offplatform) + 1.96*np.array(sem_offplatform), alpha=0.3, color = 'b')
        # plot reference line from 0 to mean_max
        ax2.plot([0,60000], [0,mean_max], 'k--')
    elif condition == '75-75':
        ax3.plot(mean_onplatform, label='on platform', color = 'r')
        ax3.fill_between(range(len(mean_onplatform)), np.array(mean_onplatform) - 1.96*np.array(sem_onplatform), np.array(mean_onplatform) + 1.96*np.array(sem_onplatform), alpha=0.3, color = 'r')
        ax3.plot(mean_offplatform, label = 'off platform', color = 'b')
        ax3.fill_between(range(len(mean_offplatform)), np.array(mean_offplatform) - 1.96*np.array(sem_offplatform), np.array(mean_offplatform) + 1.96*np.array(sem_offplatform), alpha=0.3, color = 'b')
        # plot reference line from 0 to mean_max
        ax3.plot([0,60000], [0,mean_max], 'k--')
    ax1.set(title = '0-0', ylabel='cumulative time in seconds', xlabel = 'session time in frames', ylim = (0, 850), xlim = (0, 60000))
    ax2.set(title = '0-75', ylabel='cumulative time in seconds', xlabel = 'session time in frames', ylim = (0, 850), xlim = (0, 60000))
    ax3.set(title = '75-75', ylabel='cumulative time in seconds', xlabel = 'session time in frames', ylim = (0, 850), xlim = (0, 60000))
    ax3.legend()
    # set xticks in scientific notation
    for ax in [ax1, ax2, ax3]:
        ax.ticklabel_format(style='sci', axis='x', scilimits=(0,0))
        ax.axvspan(20000, 40000, color='gray', alpha=0.1)

fig.tight_layout()
plt.show()


In [ ]:
# separate data for boxplots: AUC_onplatform_firstthird_ref, AUC_offplatform_firstthird_ref
# boxplot of timeonplatform['ABC'] by condition and by time on or off
plt.rcParams.update({'font.size': 20})
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(21,7))

colors = ['red', 'blue', 'red', 'blue']  # specify the colors you want
alpha = 0.5

import matplotlib.patches as mpatches

# Create a Patch object for each boxplot
patches = [mpatches.Patch(color=color, label=label, alpha = 0.5) for color, label in zip(colors, ['on platform', 'off platform'])]

# condition 0-0
def colored_boxplot(ax, data, position, color, alpha=0.5):
    box = ax.boxplot(data, patch_artist=True, notch=True, positions=[position], widths=0.5)
    for patch in box['boxes']:
        patch.set_facecolor(color)
        patch.set_alpha(alpha)
        patch.set_edgecolor('k')
        patch.set_linewidth(2)
    return box

# condition 0-0
ax1.axhline(1, color='k', linestyle='--')
colored_boxplot(ax1, timeonplatform[timeonplatform['condition']=='0-0']['AUC_onplatform_firstthird_ref'], 1, colors[0], alpha)
colored_boxplot(ax1, timeonplatform[timeonplatform['condition']=='0-0']['AUC_offplatform_firstthird_ref'], 2, colors[1], alpha)
colored_boxplot(ax1, timeonplatform[timeonplatform['condition']=='0-0']['AUC_onplatform_lastthird_ref'], 3, colors[2], alpha)
colored_boxplot(ax1, timeonplatform[timeonplatform['condition']=='0-0']['AUC_offplatform_lastthird_ref'], 4, colors[3], alpha)
ax1.set(title = '0-0', ylim = (0, 3.5), ylabel = 'AUC relative to diagonal', xlabel = 'session time')
ax1.set_xticklabels(['first third','', 'last third',''], ha = 'left')

# condition 0-75
ax2.axhline(1, color='k', linestyle='--')
colored_boxplot(ax2, timeonplatform[timeonplatform['condition']=='0-75']['AUC_onplatform_firstthird_ref'], 1, colors[0], alpha)
colored_boxplot(ax2, timeonplatform[timeonplatform['condition']=='0-75']['AUC_offplatform_firstthird_ref'], 2, colors[1], alpha)
colored_boxplot(ax2, timeonplatform[timeonplatform['condition']=='0-75']['AUC_onplatform_lastthird_ref'], 3, colors[2], alpha)
colored_boxplot(ax2, timeonplatform[timeonplatform['condition']=='0-75']['AUC_offplatform_lastthird_ref'], 4, colors[3], alpha)
ax2.set(title = '0-75', ylim = (0, 3.5), ylabel = 'AUC relative to diagonal', xlabel = 'session time')
ax2.set_xticklabels(['first third','', 'last third',''], ha = 'left')

# condition 75-75
ax3.axhline(1, color='k', linestyle='--')
colored_boxplot(ax3, timeonplatform[timeonplatform['condition']=='75-75']['AUC_onplatform_firstthird_ref'], 1, colors[0], alpha)
colored_boxplot(ax3, timeonplatform[timeonplatform['condition']=='75-75']['AUC_offplatform_firstthird_ref'], 2, colors[1], alpha)
colored_boxplot(ax3, timeonplatform[timeonplatform['condition']=='75-75']['AUC_onplatform_lastthird_ref'], 3, colors[2], alpha)
colored_boxplot(ax3, timeonplatform[timeonplatform['condition']=='75-75']['AUC_offplatform_lastthird_ref'], 4, colors[3], alpha)
ax3.set(title = '75-75', ylim = (0, 3.5), ylabel = 'AUC relative to diagonal', xlabel = 'session time')
ax3.set_xticklabels(['first third','', 'last third',''], ha = 'left')
ax3.legend(handles=patches) 

plt.tight_layout()
plt.show()
